In [16]:
import nest_asyncio

nest_asyncio.apply()

In [17]:
import os
from llama_index.llms.openai import OpenAI
import pandas as pd

os.environ["OPENAI_API_KEY"] = ""

In [18]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core import Settings

documents = SimpleDirectoryReader("data").load_data()
Settings.chunk_size = 512
Settings.chunk_overlap = 20

vector_query_engine = VectorStoreIndex.from_documents(
    documents,
    use_async=True
).as_query_engine()

In [19]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=vector_query_engine,
        metadata=ToolMetadata(
            name="lyft2021",
            description="lyft2021 data"
        ),
    ),
]

query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools,
    use_async=True,
)

In [20]:
df = pd.read_csv("questions/Lyft2021_queries.csv")
queries = df["Query"].tolist()

In [22]:
from llama_index.core.evaluation import FaithfulnessEvaluator, RelevancyEvaluator

llm = OpenAI(model="gpt-4o-mini", temperature=0.0)
f_evaluator = FaithfulnessEvaluator(llm=llm)
r_evaluator = RelevancyEvaluator(llm=llm)
f_scores = []
r_scores = []

for query in queries:
    response = query_engine.query(query)

    f_score = f_evaluator.evaluate_response(response=response).score * 100
    r_score = r_evaluator.evaluate_response(query=query, response=response).score * 100
    f_scores.append(f_score)
    r_scores.append(r_score)

avg_f = sum(f_scores) / len(f_scores)
avg_r = sum(r_scores) / len(r_scores)

print("Average Faithfulness Score:", avg_f)
print("Average Relevancy Score:", avg_r)

Generated 1 sub questions.
[lyft2021] Q: What were the regulatory compliance challenges faced by Lyft in 2021?
[lyft2021] A: Lyft faced regulatory compliance challenges in 2021 due to the need to work with federal, state, and local governments to ensure the availability of their ridesharing marketplace and other offerings in the United States and Canada. They encountered issues such as the adoption of new laws and regulations, changes to existing laws, permit requirements, fees, compliance obligations, and the potential for fines and penalties from various jurisdictions and governmental entities. These challenges could have adverse effects on Lyft's business, financial condition, and results of operations.
Generated 3 sub questions.
[lyft2021] Q: What were Lyft's 2021 initiatives?
[lyft2021] Q: What was Lyft's market share in 2021?
[lyft2021] Q: What were the market shares of Lyft's competitors in 2021?
[lyft2021] A: Lyft's market share in 2021 was not explicitly mentioned in the provi